# How to Use DeepBugs for Yourself
Follow along with this notebook to reproduce our replication of DeepBugs, tested on the switched-argument bug (i.e., the developer accidentally typed the arguments in reverse order.)

Or, feel free to just check out the pre-saved output - things can take a while to run.

You can also use the functions we provide to deploy DeepBugs in your own code!

## 1. Round up the source code

Start by downloading the 150k JavaScript Dataset using the links below.

* [Training Data - 10.0GB](https://1drv.ms/u/s!AvvT9f1RiwGbh6hYNoymTrzQcNA46g?e=WeJf3K)
* [Testing Data - 4.8GB](https://1drv.ms/u/s!AvvT9f1RiwGbh6hXmjPOUS-kBARjFA?e=AJY1Xf)

Save them into the `demo_data` folder.

## 2. Convert the source code ASTs to tokens
For a given corpus of code, you should have a large list of source files, each of which is converted into an Abstract Syntax Tree (AST).

In this example, we convert each AST from the 150k JavaScript Dataset into a list of tokens (e.g., "ID:setInterval" or "LIT:true"). Those lists are aggregated together into a master list of lists. This list-of-list format is important for training Word2Vec, since each list of tokens corresponds to a single source file -  tokens within a source file are closely related but tokens across source files may not be as closely related.

Example:
```
[
    [ # Corresponds to first source file
        "ID:setInterval",
        "LIT:1000",
        "ID:callbackFn",
        "LIT:true",
        "LIT:http-mode",
        ...
    ],
    [ # Corresponds to second source file
        "ID:fadeIn",
        "LIT:300",
        "ID:css",
        "LIT:color:red;margin:auto",
        ...
    ]
]
```
### Note on using our code
If you organize your ASTs into one file, such that each line of the file corresponds to one AST, you can just call our ready-to-go `ast_token_extractor.get_tokens_from_corpus()` function as shown below.

If you need more fine-grained control, you could use `ast_token_extractor.get_tokens_from()` to extract tokens from each node in a single AST.

In [10]:
from ast_token_extractor import get_tokens_from_corpus

TRAIN_DATA_PATH = "demo_data/150k_training.json"
TEST_DATA_PATH = "demo_data/150k_testing.json"

list_of_lists_of_tokens = get_tokens_from_corpus(TRAIN_DATA_PATH)

# Count the tokens extracted
num_tokens_extracted = len([len(tokens_from_single_src_file) for tokens_from_single_src_file in list_of_lists_of_tokens])

print("Extracted {0} tokens".format(num_tokens_extracted))
print("A few examples...")
print(list_of_lists_of_tokens[0])


100000it [54:32, 30.55it/s]Extracted 100000 tokens
A few examples...
['ID:gTestfile', 'LIT:regress-472450-04.js', 'ID:BUGNUMBER', 'LIT:472450', 'ID:summary', 'LIT:TM: Do not assert: StackBase(fp) + blockDepth == regs.sp', 'ID:actual', 'LIT:', 'ID:expect', 'LIT:', 'ID:test', 'ID:test', 'ID:enterFunc', 'LIT:test', 'ID:printBugNumber', 'ID:BUGNUMBER', 'ID:printStatus', 'ID:summary', 'ID:jit', 'LIT:true', 'ID:__proto__', 'ID:âœ–', 'LIT:1', 'ID:f', 'ID:eval', "LIT:for (var y = 0; y < 1; ++y) { for each (let z in [null, function(){}, null, '', null, '', null]) { let x = 1, c = []; } }", 'ID:f', 'ID:jit', 'LIT:false', 'ID:reportCompare', 'ID:expect', 'ID:actual', 'ID:summary', 'ID:exitFunc', 'LIT:test']



## Convert tokens to vectors: train a Word2Vec model
Now that you have reduced your dataset to lists of tokens, you can use them to train a Word2Vec model so that it predicts a vector for each token based on lexical similarity. In other words, a token of `LIT:true` will be lexically similar to a token of `LIT:1` but not `LIT:false`.

We train Word2Vec using the Continuous Bag of Words method with a 200-word window (i.e. for a given token, we use the previous 100 tokens and the following 100 tokens to learn the context of the token). Like the original authors, we limit the vocabulary size to the top 10,000 tokens from the dataset.

### Note on using our code
As long as you have one list of tokens per source file, aggregated into a master list of all source files, then you can call our ready-made `token2vectorizer.train_word2vec()` function as shown below.


In [11]:
from token2vectorizer import train_word2vec

WORD2VEC_MODEL_SAVE_PATH = "demo_data/word2vec.model"

model = train_word2vec(list_of_lists_of_tokens, WORD2VEC_MODEL_SAVE_PATH)

print("Should be larger difference btwn LIT:true and LIT:false", model.wv.similarity("LIT:true", "LIT:false"))
print("Should be smaller difference btwn LIT:true and LIT:1", model.wv.similarity("LIT:true", "LIT:1"))


Should be larger difference btwn LIT:true and LIT:false 0.8614448
Should be smaller difference btwn LIT:true and LIT:1 0.09773022


## Save your token-vector vocabulary for later
To speed things up when you're training and testing DeepBugs, you should save off your learned Word2Vec vocabulary in a dictionary for rapid lookup and sharing. Our `token2vectorizer.save_token2vec_vocabulary()` handles this for you in a jiffy.

Example output:
```
{
    "LIT:true": [-5.174832   -4.9506106   1.6868128   1.476279   -3.211739   ...],
    ...
}

In [12]:
import json
from gensim.models import Word2Vec
from token2vectorizer import save_token2vec_vocabulary

WORD2VEC_MODEL_READ_PATH = "demo_data/word2vec.model"
VOCAB_SAVE_PATH = "demo_data/token2vec.json"

model = Word2Vec.load(WORD2VEC_MODEL_READ_PATH)
save_token2vec_vocabulary(model, VOCAB_SAVE_PATH)

with open(VOCAB_SAVE_PATH) as example_json:
    vocab = json.load(example_json)
    print("A couple examples...")
    print("ID:Date: ", vocab["ID:Date"], "\n")
    print("ID:end: ", vocab["ID:end"])

A couple examples...
ID:Date:  [8.347164154052734, -1.7503434419631958, 2.52799654006958, 4.932645320892334, 4.3485941886901855, 2.4734597206115723, -3.104377508163452, -4.045936107635498, 7.203773498535156, -4.954057693481445, 2.7399654388427734, 7.58334493637085, -1.654502272605896, -7.7606201171875, 4.609239101409912, 0.38020211458206177, -10.710118293762207, 5.223881244659424, 2.477747917175293, 1.0023224353790283, -1.0500743389129639, -1.7351078987121582, 5.0433030128479, -4.859694004058838, -2.5254316329956055, -3.685211420059204, 3.842613697052002, -1.4842250347137451, 6.4144511222839355, -1.6959937810897827, -8.342519760131836, -8.662407875061035, -3.0137434005737305, 4.000683784484863, 6.644257068634033, 2.8766849040985107, 2.4778969287872314, 10.9281587600708, 2.503817558288574, 5.285634517669678, -3.4404542446136475, -0.23131144046783447, -3.2403600215911865, 9.892898559570312, -5.4444804191589355, -0.7952022552490234, -2.1439621448516846, 5.729167938232422, 2.91329765319824

## Generate positive/negative examples

In our example, we our testing for the switched-argument bug that the DeepBugs authors tested for, so we generate data by extracting all 2-argument function calls from the 150k dataset and then manually switching the arguments around to make "buggy" examples.

### Note on using our code
Our code is specific to switched-argument bugs. For your own bugs, you will need to write your own code to generate positive and negative training/testing examples. You can follow similar procedures to our `swarg_` scripts.

We save our examples as `.npz` files, where each file is a `Tuple[List,List]`: `(Data, Labels)`. Both `Data` and `Labels` are numpy arrays of the same length, where `Labels[i]` is 1 for positive, 0 for negative

In [13]:
import json
from swarg_gen_train_eval import gen_good_bad_fn_args
from swarg_fnargs2tokens import get_all_2_arg_fn_calls_from_ast

VOCAB_READ_PATH = "demo_data/token2vec.json"

SWARG_TRAIN_EXAMPLES_SAVE_PATH = "demo_data/switch_arg_train.npz"
SWARG_TEST_EXAMPLES_SAVE_PATH = "demo_data/switch_arg_test.npz"

gen_good_bad_fn_args(TRAIN_DATA_PATH, VOCAB_READ_PATH, SWARG_TRAIN_EXAMPLES_SAVE_PATH)
gen_good_bad_fn_args(TEST_DATA_PATH, VOCAB_READ_PATH, SWARG_TEST_EXAMPLES_SAVE_PATH)


100000it [48:05, 34.66it/s]
50000it [16:45, 49.74it/s]


(array([[  0.66330397,  -4.8157773 , -15.53928185, ...,   3.67067075,
           1.37784445,  -2.45487404],
        [  0.66330397,  -4.8157773 , -15.53928185, ...,   3.67067075,
           1.37784445,  -2.45487404],
        [-15.30671787,  -2.32782769,   5.04741478, ...,   3.74800253,
           2.01156235,  -6.74177122],
        ...,
        [ 11.28853989,  17.92754173,  -2.10642552, ...,   0.20978755,
          -6.04589081,  -2.70742297],
        [ 11.28853989,  17.92754173,  -2.10642552, ...,   0.20978755,
          -6.04589081,  -2.70742297],
        [ -0.11831304,  -1.30880451,  -0.03427336, ...,   7.0901885 ,
          -0.91956091,  -5.06203985]]),
 array([1., 1., 1., ..., 0., 0., 0.]))

## Train DeepBugs
We use examples generated from the training partition of the 150K JavaScript Dataset.

In [14]:

# TODO: Jordan and Abhi's code can just slot in here


## Test DeepBugs
We use examples generated from the test partition of the 150K JavaScript Dataset.

In [15]:
# TODO: Jordan and Abhi's code can just slot in here